In [1]:
import os
from gmsPython.gmsPy.gmsPy import *
from gmsPython.gmsWrite.gmsWrite import Syms
from gmsPython.gamY.gamY import Precompiler

*Load test database with symbols:*

In [2]:
work_folder = os.path.join(os.getcwd(),'workFolder')
data_folder = os.path.join(os.getcwd(), 'test')
db = gpyDB.GpyDB(db = os.path.join(data_folder, 'test_size1000.gdx'), ws = work_folder)

## GmsPy 

The building blocks of the model is the following:
* ```Group```: Defines group of variables. This can build on simple strings or ```gpy``` objects and conditionals as specified in ```gmsWrite```.
* ```Block```: Define blocks of equations. Name of equations are strings, but conditionals can be any that is used in ```gmsWrite``` (i.e. gpy-types).

### 1. ```Group```

Groups can be defined using four methods:
* Adding a variable + potential conditions.
* Removing a variable + potential conditions.
    * If conditions is None - this removes the entire variable from the group.
    * If conditions is in existing conditions - remove from conditions.
    * If conditions is not in existing conditions - add ```f"and not {conditions}"``` to the aggregate conditional for the variable.
* Add another group. Only works for compiled groups.
* Subtract another group. Only works for compiled groups.

#### The ```__call__``` method:

The ```__call__``` method works as follows:
1. Start by looking for entire groups to remove. We match on the group ```name``` - and not whether or not groups are actually the same compiled objects.
2. Add all ```self.v``` symbols and ```self.g``` groups to the ```self.out``` dictionary (that holds {name, condition} combinations). If ```v``` is a string, this is the symbol name , if it is a tuple the first element is the name, the second the condition.
    * If ```None``` is in ```self.out[name]``` --> replace conditions with ```None```. (This entails that the entire symbol is {zused)
3. Add all ```self.sub_v``` symbols and ```self.sub_g``` groups to the ```self.out_neg``` dictionary (that holds {name, condition} combinations). Same syntax as adding.
    * If ```None``` is in ```self.out_neg[name]``` --> remove symbol from ```self.out``` and ```self.out_neg``` dictionaries.
4. If a {name, condition} combination is in both ```self.out``` and ```self.out_neg```, the element is removed from both.



#### Examples

*Example 1: Text only:*

Specify a group only with one variable. Only use combinations of $i,j$ where the ```map[i,j]``` subset is defined. Leave out the parts where ```subset[j]``` is defined:

In [3]:
strEx = Group('ex', v = [('var[i,j]', 'map[i,j]')], sub_v = [('var[i,j]','subset[j]')])() # the () syntax compiles the group

The ```self.conditions```  produce dictionary with conditions that can be passed to ```gmsWrite.Syms.gpyCondition```: 

In [4]:
conditions = strEx.conditions

The ```self.writeVar``` method writes the variable with its conditions and other potential adjustments. For instance, this syntax gets the level attribute for the subsetted part of the variable:

In [5]:
strEx.writeVar('var[i,j]', c = conditions, l='.l')

'var.l[i,j]$((map[i,j] and ( not (subset[j]))))'

The ```self.declare``` method returns a block of text that declares variables from the group:

In [6]:
print(strEx.declare())

variables
	var[i,j]
;




We can add exceptions to the declare statement. In this case, as the variable $var[i,j]$ is the only variable in the group, adding this as an exception means returning an empty string instead:

In [7]:
print(strEx.declare(exceptions = ['var[i,j]'])) # the lines of text with "variables" and ";" are removed if no variables are declared as is the case here

*Note: The text only example means that other features such as merging of gams groups and subsetting the Python version of the symbols does not work. If we only use string methods, we may as well simply write the entire text strings ourselves.*

*Example 2:* Initialize with entire two variables. Remove one of them again entirely and a subset of the other: 

In [8]:
ex2 = Group('ex', v = ['var', 'var1d'], sub_v = [('var', db['subset']), 'var1d'])()
conditions = ex2.conditions
ex2.writeVar('var', db = db, c = conditions, l = '.l') # get variable

'var.l[i,j]$(( not (subset[j])))'

*Declare group:*

In [9]:
print(ex2.declare(db = db) )

variables
	var[i,j]
;




*Example 3: Add/subtract other compiled groups. Here, add the group above* 

In [10]:
ex3 = Group('ex3', v = None, sub_v = [('var', db['map'])], g = [ex2])()
conditions = ex3.conditions
ex3.writeVar('var', db = db, c = conditions, l = '.l') #

'var.l[i,j]$((( not (subset[j])) and ( not (map[i,j]))))'

*Declare group:*

In [11]:
print(ex3.declare(db = db) )

variables
	var[i,j]
;




*Example 4:*

In [12]:
ex2 = Group('ex', v = ['var', 'var1d'])()
conditions = ex2.conditions
ex2.writeVar('var', db = db, c = conditions)

'var[i,j]'

In [13]:
ex3 = Group('ex3', v = None, sub_v = [('var', db['map'])], g = [ex2])

### 2. ```jTerms```